In [1]:
import logging

from data_collection.models import Game, Round
from data_collection.questions_candidates import get_question_candidates
from data_collection.questions_pick import pick_question
from data_collection.semaphore import set_semaphore
from data_collection.guess import guess
from data_collection.answer import answer
from data_collection.simulate_game import run_game


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

from dotenv import load_dotenv
from openai import AsyncOpenAI

from data_collection.utils import call_gpt4, dataset

load_dotenv()

True

In [2]:
set_semaphore("gpt-4", 10)

In [3]:
client = AsyncOpenAI()

In [6]:
game = await run_game(client, "sweet potato")

Game sweet potato, round 0


2024-07-05 22:54:26,858 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Tokens used: 321


2024-07-05 22:54:28,539 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Tokens used: 491


...

Tokens used: 789


2024-07-05 22:56:33,277 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Tokens used: 693


2024-07-05 22:56:35,048 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Tokens used: 890


In [22]:
print(dataset[-1]['response'])

Given the history of questions and answers, we know that the keyword is a non-living, naturally occurring plant that is a type of vine, commonly found in tropical regions, used in traditional medicine, known for its large leaves, and native to South America. Previous guesses like "cats claw" and "chayote vine" have been made, but none were correct. Considering these characteristics, another well-known vine fitting this description is the "Jungle Grape" or "Cissus verticillata."

Cissus verticillata


In [ ]:
len(dataset)

In [7]:
# response = await call_gpt4(client, "Ask your first question", temperature=0.1, request_type="test", game_state=Game())

2024-07-05 22:28:28,485 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Tokens used: ChatCompletion(id='chatcmpl-9hh1QZRBB34qJnlHokBjgNTuzzJ6N', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Is the keyword a place?', role='assistant', function_call=None, tool_calls=None))], created=1720200508, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_ce0793330f', usage=CompletionUsage(completion_tokens=6, prompt_tokens=380, total_tokens=386))


In [4]:
game = Game(
    rounds=[
        Round(
            question="Is it a living object?",
            answer="yes"
        ),
        Round(
            question="Is it edible?",
            answer="yes"
        ),
        Round(
            question="Is it a fruit?",
            answer="no"
        ),
        Round(
            question="Is it an animal?",
            answer="no"
        ),
        Round(
            question="Is it a dairy product?",
            answer="no"
        ),
        Round(
            question="Is it a vegetable?",
            answer="yes"
        ),
        Round(
            question="Is it commonly green in color?",
            answer="no"
        ),
        Round(
            question="Is it commonly used in salads?",
            answer="no",
        ),
        Round(
            question="Is it commonly red in color?",
            answer="no",
        ),
        Round(
            question="Is it commonly cooked before eating?",
            answer="yes",
        ),
        # Round(
        #     question="Is it commonly used in soups?",
        #     answer="yes",
        #     guess="pumpkin",
        # ),
        Round(
            question="Is it commonly orange in color?",
            answer="yes",
            guess="carrot"
        )
    ]
)

In [5]:
candidates = await get_question_candidates(client, game)

2024-07-05 22:43:47,818 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Tokens used: 651


In [6]:
candidates

['Is it commonly used in soups or stews? (60% - 40%)',
 'Is it commonly used in desserts? (10% - 90%)',
 'Is it a root vegetable? (80% - 20%)',
 'Is it commonly used in baby food? (50% - 50%)',
 'Is it commonly used in Asian cuisine? (40% - 60%)',
 'Is it commonly used in Italian cuisine? (30% - 70%)',
 'Is it high in beta-carotene? (90% - 10%)',
 'Is it commonly used in juices? (20% - 80%)',
 'Is it commonly used in Thanksgiving dishes? (70% - 30%)',
 'Is it commonly used in French cuisine? (40% - 60%)']

In [138]:
candidates_scored = await get_question_candidates_scored(client, game, candidates)

2024-07-05 21:09:08,387 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [139]:
candidates_scored

['Is it a root vegetable? (90% - 10%)',
 'Is it commonly used in making purees? (70% - 30%)',
 'Is it commonly used in making baby food? (60% - 40%)',
 'Is it commonly used in making juices? (20% - 80%)',
 'Is it a tuber? (50% - 50%)',
 'Is it commonly used in making pies? (40% - 60%)',
 'Is it commonly used in making fries? (30% - 70%)',
 'Is it commonly used in making casseroles? (50% - 50%)',
 'Is it commonly used in making curries? (30% - 70%)',
 'Is it commonly used in stews? (80% - 20%)']

In [140]:
reasoning, question = await pick_question(client, game, candidates_scored)

2024-07-05 21:09:11,411 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [141]:
reasoning

'We know that the object is an edible, non-fruit, non-animal, non-dairy vegetable that is not commonly green or red, is commonly cooked before eating, and is commonly used in soups and is orange in color. Given these constraints, the vegetable is likely to be something like a carrot or a pumpkin. To narrow it down further, we should ask a question that splits the remaining possibilities effectively. The question "Is it a root vegetable?" is highly informative because it will help us distinguish between root vegetables like carrots and non-root vegetables like pumpkins. '

In [142]:
question

'Is it a root vegetable?'

In [143]:
reasoning, guessed_keyword = await guess(client, game)

2024-07-05 21:09:14,441 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [144]:
print(reasoning)

The keyword is a vegetable that is not green or red, is commonly cooked before eating, and is often used in soups. It is also commonly orange in color. Given these characteristics, and considering that both pumpkin and carrot have been guessed already, another common orange vegetable that fits these criteria is sweet potato. 


In [145]:
guessed_keyword

'sweet potato'

In [153]:
reasoning, answer_str = await answer(client, game, "carrot", "Is it commonly used in sex?")

2024-07-05 21:28:51,702 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [154]:
print(reasoning)

Carrots are not commonly associated with sexual use. While there might be rare instances or jokes about it, it is not a common or typical use for carrots. 


In [156]:
print(answer_str)

no
